# SCC-ICMC-USP - 2o. semestre de 2021
# SCC0275 - Exercício 6

### Profa. Roseli A. F. Romero

### Monitor: Kenzo Sakiyama

Nro do grupo:

Alunos:


1.   Stefan Taiguara Couperus Leal - 10414866


## Parte 1 - Medidas de avaliação e bootstrap

---


### Carregando dados e definindo função de preprocessamento

Para as próximas questões, utilizaremos o dataset **breast_cancer** disponibilizado pelo sklearn. A descrição do conjunto de dados e de seus atributos será apresentada a seguir (atente-se ao atributo alvo "class").


In [69]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df["class"] = data.target
data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [70]:
print(data["DESCR"])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [78]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix

def preprocess(x_treino, x_teste, y_treino, y_teste):

    scaler = MinMaxScaler()
    # Computando estatísticas dos dados de treino
    scaler.fit(x_treino)
    # Transformando dados com a normalização MinMax
    x_treino_novo = scaler.transform(x_treino)
    x_teste_novo = scaler.transform(x_teste)

    # Não é necessário alterar os labels a menos que seja feito um tipo de amostragem (over/under sampling)
    y_treino_novo = y_treino
    y_teste_novo = y_teste

    return x_treino_novo, x_teste_novo, y_treino_novo, y_teste_novo


def accuracy(conf_mat):
    return (sum(np.diagonal(conf_mat))) / (conf_mat.sum())


# confusion_matrix(y_test, y_pred)

df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


### Questão 01. 

-  Implemente a técnica bootstrap utilizando a função abaixo
- Sugestão, utilize o material de apoio como referência

In [83]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from numpy import mean, std
from matplotlib import pyplot as plt
import numpy as np

def classificacao_bootstrap(data, columns, target, preproc_fn, score_fn, score_name, conf_matrix_fn, 
                   no_bs=1000, p_teste=0.2, plot=True):
    """
    Executa classificação do conjunto de dados passado
    ---------------------------------------------------------------
    data:       DataFrame. Conjunto de dados
    columns:    Lista de inteiros. Índice das colunas utilizadas no treinamento e teste
    target:     Inteiro. Índice da coluna alvo
    preproc_fn: Função. Faz o pré-processamento da base já separada em treino e teste 
    score_fn:   Função. A função que calcula a medida de desempenho desejada. Deve ser uma 
                função que compara dois vetores, o primeiro vetor são os valores preditos
                pelo classificador, o segundo os rótulos reais
                Vide exemplo das funções em 
                http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
                como por exemplo, sklearn.metrics.accuracy_score
                http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    score_name: String. Uma string com o nome da medida de desempenho
    conf_matrix_fn: Função. Calcula matriz de confusão 
    folds:      Inteiro. Número de folds na validação cruzada
    plot:       Booleano. True para plotar os gráficos False para não plotar
    ---------------------------------------------------------------
    Realiza a classificação em 6 modelos (perceptron, 
    SVM com kernel polinomial de grau 3, Árvore de decisão, 3NN, 5NN, e 7NN)
    Plot o gráfico de desempenho para cada classificador.
    Retorna um dicionário com os classiff    """
    # inicializa os modelos com os parâmetros solicitados
    prcp = Perceptron()
    dt = DecisionTreeClassifier(criterion='gini', splitter='best', min_samples_split=int(len(data)*0.1))
    
    clfs = [prcp, dt]
    clfs_names = ['perceptron', 'dt']
    
    #Inicializa estruturas para matrizes de confusão 
    confusion_matrices = {
        'perceptron': np.zeros((2,2)),
        'dt': np.zeros((2,2)),
    }
    
    results = {'perceptron':[], 'dt':[]}
    df = data
    
    for c, c_name in zip(clfs, clfs_names):
        no_exemplos = df.shape[0]
        p_treino = 0.8
        no_bootstrap = 1000

        matriz_conf = np.array([[0,0], [0,0]])
        
        for i in range(0, no_bootstrap):
            amostra = data.sample(n = no_exemplos, replace=True) # Obtendo amostra
            print(f"X_train shape: {amostra.columns}")
            
            
            X = amostra.iloc[:, 0:-2]
            y = amostra.iloc[:, -1]

            X_train, X_test, y_train, y_test = train_test_split(amostra,
                                                                target, 
                                                                stratify=target,
                                                                test_size=p_teste)
            
            
            ss = StandardScaler()
            X_train = ss.fit_transform(X_train)
            X_test = ss.transform(X_test)

            data_train = pd.DataFrame(data=np.c_[X_train, y_train],
                                      columns=['feature_names' ,'target'])
            classe_0 = data_train[data_train['target'] == 0]
            classe_1 = data_train[data_train['target'] == 1]

            # Coletando o número de exemplos da classe 0 para superamostragem da classe 1  
            n_elementos_classe_0 = data_train['target'].value_counts()[0] 
            # Superamostragem da classe 1 para numero de elementos da classe 0
            data_train = pd.concat([classe_0, classe_1.sample(n=n_elementos_classe_0, replace=True)])

            X_train, y_train = data_train.iloc[:, 0:4], data_train.iloc[:, -1]

            clf = c.fit(X=X_train, y=y_train)

            # valores preditos pelo classificador
            y_pred = clf.predict(x_test)
            # rótulos verdadeiros convertidos para array
            y_test = np.array(y_test)
            conf = cm(y_test, y_pred)

            results[c_name][score_name].append(score_fn(conf))

            resultado = score_fn( y_test, y_pred)
            results[c_name].append(resultado)

            matriz_conf += confusion_matrix(y_test, y_pred, labels=[0, 1])
      ################# IMPLEMENTE AQUI O BOOTSTRAP #########################
        #    Dentro desse loop:
        #     c: variável-objeto que representa o classificador
        #     c_name: Nome do classificador
        #     results: dicionário com vetores de resultado para cada classificador
      #######################################################################        
     
      #####################  ###############################################
    if not plot:
        return {'results': results, 'clfs':clfs}
    # faz o plot de desempenho dos classificadores
    plt.figure(figsize=(8,8))
    plt.bar(range(1, len(clfs)+1), [mean(results[name]) for name in clfs_names], 
                                yerr=[std(results[name]) for name in clfs_names])
    plt.xticks(range(1, len(clfs)+1), clfs_names, rotation=45)
    title = 'Desempenho dos classificadores - %s'%(score_name)
    plt.title(title)
    plt.show()
    
    return {'results': results, 'clfs':clfs, 'confusion_matrices': confusion_matrices}

In [84]:
results = classificacao_bootstrap(df, data.data, data.target, preprocess, accuracy, "accuracy", confusion_matrix, 
                   no_bs=1000, p_teste=0.2, plot=True)


X_train shape: Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'class'],
      dtype='object')


ValueError: Shape of passed values is (455, 32), indices imply (455, 2)

### Questão 02. 

- Rode a função 4 vezes utilizando 2 métricas diferentes do **sklearn.metrics** (2 execuções por métrica);
- Compare os resultados e discuta as vantagens/desvantagens de cada uma das métricas

In [ ]:
# Seu codigo aqui
import sklearn.metrics

## Parte 2 - Curva ROC e Teste de hipótese

## Funções novas utilizadas no exercício

- `pandas.Series.nunique()` ([link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nunique.html)): Conta quantidade de valores únicos de uma coluna. Útil para verificar se uma coluna é relevante ou não
- `scipy.stats.ttest_ind()` ([link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind)): Calcula o teste t para duas amostras independentes
- `sklearn.metrics.plot_roc_curve()` ([link](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_roc_curve.html)): Plota a curva ROC de um classificador dado um conjunto de input e alvo

### Questão 01.

Faça a exploração dos dados. Isto é, carregue, substitua valores faltantes, padronize os dados, etc. Faça também a seleção dos atributos que achar mais relevantes. 

 - Dica: Utilize a função `nunique()` durante a exploração dos dados. Você pode utilizar o "bom senso" (além de outras ferramentas é claro) na hora de escolher qual atributo do conjunto manter

In [87]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("titanic.csv")
# df.info()
df.dropna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [90]:
# Etapas de pré-processamento que podem ser feitas antes de separar em treino/teste vão aqui
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
#        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked

features_categoricos = ['Name', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Ticket', 'Cabin']
features_numericos = ['PassengerId', 'Survived', 'Pclass', 'Age',  'Fare']


def transformar_base(df, features_numericos, features_categoricos):
    
    # Criando os pipelines
    pipeline_numerico = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    pipeline_categorico = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder())])

    # Criando a transformação do conjunto de dados:
    transformacao = ColumnTransformer(
        transformers=[
            ('transformacao numerica', pipeline_numerico, features_numericos),
            ('transformacao categorica', pipeline_categorico, features_categoricos),        
        ])
    # Aplicando a transformação no dataset:
    dados_transformados = transformacao.fit_transform(df)
    
    return dados_transformados

dados_transformados = transformar_base(df, features_numericos, features_categoricos)
pd.DataFrame(dados_transformados)

,0
0,"(0, 0)\t-1.730107956920867\n (0, 1)\t-0.789..."
1,"(0, 0)\t-1.7262200738716067\n (0, 1)\t1.266..."
2,"(0, 0)\t-1.7223321908223463\n (0, 1)\t1.266..."
3,"(0, 0)\t-1.718444307773086\n (0, 1)\t1.2669..."
4,"(0, 0)\t-1.7145564247238256\n (0, 1)\t-0.78..."
...,...
886,"(0, 0)\t1.7145564247238256\n (0, 1)\t-0.789..."
887,"(0, 0)\t1.718444307773086\n (0, 1)\t1.26698..."
888,"(0, 0)\t1.7223321908223463\n (0, 1)\t-0.789..."
889,"(0, 0)\t1.7262200738716067\n (0, 1)\t1.2669..."


In [38]:
# Etapas de pré-processamento que serão feitas após os dados estarem divididos vão na função
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np



# Balanceando no conjunto de treino 

classe_0 = df[df['Survived'] == 0]
classe_1 = df[df['Survived'] == 1]
df = pd.concat([classe_0, classe_1.sample(n=207, replace=True)])

X_train, y_train = df.iloc[:, 0:4], df.iloc[:, -1]


df_new["Survived"].value_counts()


x_treino, x_teste, y_treino, y_teste = train_test_split(dados_transformados, 
                                                        test_size = 0.3,random_state = 0, 
                                                        stratify = dados_transformados.toarray())


def preprocess(X_train, X_test, y_train, y_test, features_categoricos, features_numericos):
    return X_modf_train, X_modf_test, y_train, y_test

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

### Questão 02.

Separe o conjunto de dados de maneira estratificada (através do parâmetro `stratify` da função `train_test_split`) em 20% para teste e 80% para treino. Depois plote a curva ROC (`sklearn.metrics.plot_roc_curve`) para **todos** os classificadores (no mesmo gráfico). 

Os melhores classificadores da questão anterior também apresentaram melhor desempenho na curva ROC? O que pode ter ocorrido? Teste diferentes valores de `random_state` na função `train_test_split` e observe o comportamento das curvas.


*   Dica: Para plotar múltiplas curvas ROC no mesmo gráfico, defina uma figura com `fig, ax = plt.subplots()` e passe `ax` como parâmetro da função `plot_roc_curve`. Não se esqueça de passar também o nome do classificador para que o seu gráfico fique mais fácil de interpretar



In [88]:
# Separar conjunto e pré processamento
df_novo = df.copy(deep=True)
y = df_novo["Survived"]
del df_novo['Survived']
X = df_novo
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [89]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    test_size = 0.2,
                                                    random_state = 0, 
                                                    stratify = df)


ValueError: Input contains NaN

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, metrics
import numpy as np

classificadores = {
  "Perceptron" : {"modelo": Perceptron(), "scores": []},
  "Multi-Layer Perceptron (15,)" : {"modelo": MLPClassifier(random_state=1, hidden_layer_sizes=(15,), max_iter=2000), "scores": []},
  "Árvore Decisão Critério Gini" : {"modelo": DecisionTreeClassifier(criterion='gini'), "scores": []},  
  "KNN k=5" : {"modelo": KNeighborsClassifier(n_neighbors=5), "scores": []}
}

plt.rcParams['figure.figsize'] = [15, 10]
fig, ax = plt.subplots()
for classificador_name in classificadores:  
 # Seu código aqui:
 # 1) Preprocessamento
 # 2) Fit
 # 3) Predict
 # 4) Avaliação com plot_roc_curve
plt.show() 

### Questão 03.

Implemente o 10-Fold Cross Validation (pode usar o scikit) com os dois melhores classificadores de acordo com a curva ROC e guarde a acurácia de cada fold na chave 'scores' do dicionário de classificadores.
- Sugestão: utilize o gabarito do Ex7 como referência para implementação.

In [ ]:
# Seu código aqui
# Lembre-se do pré-processamento

### Questão 04.

Verifique se há diferença estatística significante entre suas acurácias da questão anterior utilizando o teste T (`scipy.stats.ttest_ind`). Considere que há diferença significante se p <= 0.05 (rejeita-se a hipótese nula)

In [ ]:
from scipy.stats import ttest_ind
# Seu código aqui